In [1]:
!pip install torch accelerate bitsandbytes datasets transformers peft trl scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.6 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
from torch import cuda, bfloat16
import transformers

import torch
import torch.nn as nn

In [4]:
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

In [5]:
from peft import LoraConfig, get_peft_model
modules = ['v_proj', 'up_proj', 'down_proj', 'k_proj', 'o_proj', 'q_proj', 'gate_proj']
config = LoraConfig(
    r=16,  #attention heads
    lora_alpha=64,  #alpha scaling
    target_modules=modules,  #gonna train all
    lora_dropout=0.1,  # dropout probability for layers
    bias="none",
    task_type="CAUSAL_LM", #for Decoder models like GPT Seq2Seq for Encoder-Decoder models like T5
)

In [6]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "Venkat-Ram-Rao/Llama2_7B_qlora_CoT_FT-v2"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
mem = model.get_memory_footprint()
print("Memory footprint: {} ".format(mem))

#7,227,846,656

Memory footprint: 7227846656 


In [11]:
def get_answer(txt):
  batch = tokenizer(tst, return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=90)

  #print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

  output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
  end_index = output.find("### End")
  output = output[:end_index]

  return(output)


In [13]:
tst = """Read the Instruction below and provide an answer.

### INSTRUCTION:
In this task, you are given an input list A. You need to find all the elements of the list that are numbers and calculate their sum.

['i', '100', 'h', '849', 'e']



### RESPONSE:"""

print(get_answer(tst))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Read the Instruction below and provide an answer.

### INSTRUCTION:
In this task, you are given an input list A. You need to find all the elements of the list that are numbers and calculate their sum.

['i', '100', 'h', '849', 'e']



### RESPONSE:
The elements that are numbers are '849' and '849'.


Therefore the answer is 849






In [14]:
tst = """Read the Instruction below and provide an answer.

### INSTRUCTION:
Bob is 25 years old ans was born in Syndney, Australia.

Where was Bob born?



### RESPONSE:"""

print(get_answer(tst))

Read the Instruction below and provide an answer.

### INSTRUCTION:
Bob is 25 years old ans was born in Syndney, Australia.

Where was Bob born?



### RESPONSE:
The sentence states that Bob was born in Sydney, Australia.


Therefore the answer is Sydney, Australia






In [16]:
tst = """Read the Instruction below and provide an answer.

### INSTRUCTION:
Bob is 25 years old and was born in Syndney, Australia.

From the information, can you tell how tall Bob is?



### RESPONSE:"""

print(get_answer(tst))

Read the Instruction below and provide an answer.

### INSTRUCTION:
Bob is 25 years old ans was born in Syndney, Australia.

From the information, can you tell how tall Bob is?



### RESPONSE:
The information states that Bob is 25 years old.


Therefore the answer is 25






In [18]:
tst = """Read the Instruction below and provide an answer.

### INSTRUCTION:
Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.
When did Virgin Australia start operating?


### RESPONSE:"""

print(get_answer(tst))

Read the Instruction below and provide an answer.

### INSTRUCTION:
Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.
When did Virgin Australia start operating?


### RESPONSE:
Virgin Australia started operating on 31 August 2000.


Therefore the answer is 31 August 2000






In [22]:
tst = """Read the Instruction below and provide an answer.

### INSTRUCTION:
Product X is used to treat migraines and nausea. It is well tolerated in patients above 15 years of age. Adverse Events include some rashes and restless leg syndrome.
What can Product X be used to treat?


### RESPONSE:"""

print(get_answer(tst))

Read the Instruction below and provide an answer.

### INSTRUCTION:
Product X is used to treat migraines and nausea. It is well tolerated in patients above 15 years of age. Adverse Events include some rashes and restless leg syndrome.
What can Product X be used to treat?


### RESPONSE:
The question states that Product X is used to treat migraines and nausea.


Therefore the answer is migraines and nausea






In [23]:
tst = """Read the Instruction below and provide an answer.

### INSTRUCTION:
Product X is used to treat migraines and nausea. It is well tolerated in patients above 15 years of age. Adverse Events include some rashes and restless leg syndrome.
I am 9 years old. Can I use Product X?


### RESPONSE:"""

print(get_answer(tst))

Read the Instruction below and provide an answer.

### INSTRUCTION:
Product X is used to treat migraines and nausea. It is well tolerated in patients above 15 years of age. Adverse Events include some rashes and restless leg syndrome.
I am 9 years old. Can I use Product X?


### RESPONSE:
The answer is "no". The product is not recommended for children below 15 years of age.


Therefore the answer is no






In [24]:
tst = """Read the Instruction below and provide an answer.

### INSTRUCTION:
Product X is used to treat migraines and nausea. It is well tolerated in patients above 15 years of age. Adverse Events include some rashes and restless leg syndrome.
Can Product X be used to treat knee pain?


### RESPONSE:"""

print(get_answer(tst))

Read the Instruction below and provide an answer.

### INSTRUCTION:
Product X is used to treat migraines and nausea. It is well tolerated in patients above 15 years of age. Adverse Events include some rashes and restless leg syndrome.
Can Product X be used to treat knee pain?


### RESPONSE:
The question states that Product X is used to treat migraines and nausea. It is well tolerated in patients above 15 years of age. Adverse Events include some rashes and restless leg syndrome.


Therefore the answer is no






In [28]:
tst = """Read the Instruction below and provide an answer.

### INSTRUCTION:
Product X is used to treat migraines and nausea. It is well tolerated in patients above 15 years of age. Adverse Events include some rashes and restless leg syndrome.
I have nausea. Can I use Product X?


### RESPONSE:"""

print(get_answer(tst))

Read the Instruction below and provide an answer.

### INSTRUCTION:
Product X is used to treat migraines and nausea. It is well tolerated in patients above 15 years of age. Adverse Events include some rashes and restless leg syndrome.
I have nausea. Can I use Product X?


### RESPONSE:
The answer is "no". The question states that Product X is used to treat migraines and nausea. However, the answer provided is "no" because the patient is below 15 years of age.


Therefore the answer is no




